# Reading stored Streaming-Data and display graph

Load Sentiment and Bitcoint-Price from Parquet file. Prepare the Dataframe with Koalas and display graph with matplotlib

In [0]:
#!pip install koalas
# Databricks has this already included

In [0]:
import pandas as pd
import numpy as np
import databricks.koalas as ks
import matplotlib.pyplot as plt
from databricks.koalas.config import set_option, reset_option
set_option("compute.ops_on_diff_frames", True)

In [0]:
# Load and sort parquet files
tweet_df = sqlContext.read.parquet("dbfs:/FileStore/original/tweet_sparksql")
tweet_df = tweet_df.sort('t_stamp')

# transform dataframe to Koalas in order to use all the pandas function (like 'cumsum')
tweet_df = ks.DataFrame(tweet_df)
tweet_df.head(5)

Out[2]:

,window,count_tweets,sub_avg,pol_avg,window_price,t_stamp
0,"{'start': 2021-06-08 14:43:00, 'end': 2021-06-...",15,0.298358,0.082500,31815.279297,2021-06-08 14:43:05
1,"{'start': 2021-06-08 14:43:06, 'end': 2021-06-...",38,0.322026,0.084145,31815.279297,2021-06-08 14:43:11
2,"{'start': 2021-06-08 14:43:12, 'end': 2021-06-...",36,0.379156,0.115807,31815.279297,2021-06-08 14:43:17
3,"{'start': 2021-06-08 14:43:18, 'end': 2021-06-...",45,0.271342,0.143319,31815.279297,2021-06-08 14:43:23
4,"{'start': 2021-06-08 14:43:24, 'end': 2021-06-...",43,0.303333,0.133831,31815.279297,2021-06-08 14:43:29


In [0]:

graph_df = ks.DataFrame(tweet_df.pol_avg.cumsum().alias('sentiment'))
graph_df['price'] = tweet_df.window_price.div(30000)
graph_df['t_stamp'] = tweet_df.t_stamp
graph_df

/databricks/python/lib/python3.8/site-packages/databricks/koalas/series.py:993: FutureWarning: Series.alias is deprecated as of Series.rename. Please use the API instead.
 warnings.warn(
Out[8]:

,sentiment,price,t_stamp
0,0.082500,1.060509,2021-06-08 14:43:05
1,0.166645,1.060509,2021-06-08 14:43:11
2,0.282452,1.060509,2021-06-08 14:43:17
3,0.425771,1.060509,2021-06-08 14:43:23
4,0.559602,1.060509,2021-06-08 14:43:29
5,0.667930,1.060509,2021-06-08 14:43:35
6,0.780687,1.060509,2021-06-08 14:43:41
7,0.923804,1.060509,2021-06-08 14:43:47
8,1.017777,1.060509,2021-06-08 14:43:53
9,1.161170,1.060509,2021-06-08 14:43:59


In [0]:
kdf = ks.DataFrame({'sentiment': graph_df['sentiment'].to_numpy(),
                    'price': graph_df['price'].to_numpy()},
                   index=graph_df['t_stamp'].to_numpy())
kdf.plot.line()

Out[20]:

In [0]:
#plot 1:
x = np.array([0, 1, 2, 3])
y = np.array([3, 8, 1, 10])

plt.subplot(1, 2, 1)
plt.plot(x,y)

#plot 2:
x = np.array([0, 1, 2, 3])
y = np.array([10, 20, 30, 40])

plt.subplot(1, 2, 2)
plt.plot(x,y)

plt.show()

In [0]:
#tweet_df.columns
tweet_df.select("pol_avg", "window_price").describe().show()

In [0]:
# Register as a table to run sql queries - not used here, we go with dataframes
# tweet_df.registerTempTable("tweets")
# sqlContext.sql("select * from tweets").collect()

In [0]:
# Functions that return a sentiment and a price datapoint for each window. This function will be repetedly called.
import random

def sentiment():
  s = [random.random()]
  return s

def price():
  p = [random.random()]
  return p

In [0]:
sentiment()

In [0]:
# Initialize graph
fig = plt.figure(figsize=(10, 3))
ax1 = fig.add_subplot(111)
ax1.set_xlim(0, 600)
ax1.set_ylim(-1, 1)

# Create a StreamAnimation object
anim = StreamAnimation(fig, interval=100)  

s_stream = Stream(ax1, sentiment)
p_stream = Stream(ax1, price)
anim.add_stream(s_stream, p_stream)

plt.tight_layout()
plt.show()
